In [1]:
import gc
import os
from datetime import timedelta
from typing import Dict, Any

import cv2
import numpy as np
import torch
import torch.multiprocessing as mp
import torch.distributed as dist
from fire import Fire
from omegaconf import OmegaConf
from pytorch_toolbelt.utils.distributed import is_main_process
from pytorch_toolbelt.utils import fs
from tqdm import tqdm

from xview3 import *
from xview3.centernet.visualization import create_false_color_composite, vis_detections_opencv
from xview3.constants import PIX_TO_M
from xview3.inference import (
    predict_multilabel_scenes,
)

from submit_multilabel_ensemble import *

/home/ibrahm/miniconda3/envs/xview3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [6]:
configs = 'configs/inference/1128_b5_b4_vs2_fliplr.yaml'
world_size = 1
master_addr = 'localhost'
master_port = find_free_port()
# os.environ['MASTER_ADDR'] = master_addr
# os.environ['MASTER_PORT'] = master_port
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
data_dir=os.environ.get("XVIEW3_DIR", "g:/xview3" if os.name == "nt" else "/home/bloodaxe/data/xview3")
local_rank=int(os.environ.get("LOCAL_RANK", 0))
world_size=2 #int(os.environ.get("WORLD_SIZE", 1))

if world_size > 1:
        # torch.distributed.init_process_group(backend="gloo",world_size =2,rank = 0)
        # mp.spawn(setup_process,args=(master_addr,master_port,world_size), nprocs = 1)
        mp.spawn(setup_process,args=(master_addr,master_port,world_size), nprocs = world_size)
        # torch.cuda.set_device(local_rank)
        print("Initialized distributed inference", local_rank, world_size)



setting up rank=0 world_size=2 backend='nccl'
master_addr='localhost' master_port='38371'
setting up rank=1 world_size=2 backend='nccl'
master_addr='localhost' master_port='38371'
rank=0 init complete
rank=1 init complete
Initialized distributed inference 0 2


In [12]:
data_dir='/media/ibrahm/SSD2TB870EV/xViewSARData/xView3dataForWinner'
config = OmegaConf.load(configs)
data = XView3DataModule(data_dir)

model, checkpoints, box_coder = ensemble_from_config(config)

checkpoint = checkpoints[0]
normalization_op = build_normalization(checkpoint["checkpoint_data"]["config"]["normalization"])
channels = checkpoint["checkpoint_data"]["config"]["dataset"]["channels"]

    # _, _, holdout_df, shore_root = data.train_val_split(
    #     splitter=checkpoint["checkpoint_data"]["config"]["dataset"]["splitter"],
    #     fold=checkpoint["checkpoint_data"]["config"]["dataset"]["fold"],
    #     num_folds=checkpoint["checkpoint_data"]["config"]["dataset"]["num_folds"],
    # )

test_scenes = np.array(data.get_test_scenes())

channels_last = config["inference"]["channels_last"]
tile_size = config["inference"]["tile_size"]
tile_step = config["inference"]["tile_step"]
tta_mode = config["ensemble"]["tta"]

submission_dir = config["submission_dir"]
os.makedirs(submission_dir, exist_ok=True)

Applying sigmoid activation to ['CENTERNET_OUTPUT_OBJECTNESS_MAP', 'CENTERNET_OUTPUT_VESSEL_MAP', 'CENTERNET_OUTPUT_FISHING_MAP'] after each model 12
Wrapping models with TTA fliplr


In [19]:
print('scene: ',test_scenes)
print('last channels: ',channels_last) 
print('tile size: ',tile_size) 
print('tile step: ',tile_step) 
print('tta mode: ',tta_mode) 
print('sub directory: ',submission_dir) 

scene:  ['/media/ibrahm/SSD2TB870EV/xViewSARData/xView3dataForWinner/test/05bc615a9b0e1159t'
 '/media/ibrahm/SSD2TB870EV/xViewSARData/xView3dataForWinner/test/72dba3e82f782f67t']
last channels:  False
tile size:  2048
tile step:  1536
tta mode:  fliplr
sub directory:  submissions/1128_b5_b4_vs2_fliplr


In [ ]:

if config["inference"]["use_traced_model"]:
    traced_model_path = os.path.join(submission_dir, "traced_ensemble.jit")
    if os.path.exists(traced_model_path):
        model = torch.jit.load(traced_model_path)
    else:
        with torch.no_grad():
            if channels_last:
                model = model.to(memory_format=torch.channels_last)
                print("Using channels last format")

            model = torch.jit.trace(
                model,
                example_inputs=torch.randn(1, len(channels), tile_size, tile_size).cuda(),
                strict=False,
            )
            if is_main_process():
                torch.jit.save(model, traced_model_path)

del checkpoints
gc.collect()

prefix = "test_"
suffix = f"_step_{tile_step}_tta_{tta_mode}"

test_predictions_dir = os.path.join(submission_dir, f"{prefix}{suffix}")
os.makedirs(test_predictions_dir, exist_ok=True)